In [1]:
import json
import os
import geopandas as gpd
import numpy as np
from shapely.geometry import MultiLineString, Point, LineString
import math

In [2]:
user_name = 'baoyuliu'

# create a path to the GeoJSON file
root = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand')
fn = 'Roads_small.geojson'
full_path = os.path.join(root, fn)

# use geopandas to read in the GeoJSON file
roads = gpd.read_file(full_path)

# create a new column that contains the length of each road
viz_roads = roads.copy()
print(viz_roads.head())

#check the data type of the geometry column
print(viz_roads['geometry'].dtype)
print(viz_roads.type)

   fid  OBJECTID                                DKM_UUID  OBJEKTART  NAME  \
0   95      2126  {84F4C50E-E009-4DB8-AF3C-DEE20F3177F6}       1700  None   
1  113      2145  {6408AD08-C7B5-4602-81D7-A9B7FBFFD5FF}       1700  None   
2  114      2146  {578A6D84-CC7B-4687-B7C6-79313525392A}       1700  None   
3  163      2195  {BF0A1AEF-817C-45DA-A33C-6C37903D19BB}       1500  None   
4  169      2201  {C3EB4DD7-7E8F-4BB5-9DFD-BEE78567F8A7}       1400  None   

  ROUTENNUMMER   SG  STUFE  KUNSTBAUTE  BEFAHRBARKEIT  ...  OVERRIDE1 OFF2  \
0         None  200      0         100            -99  ...       None  NaN   
1         None  200      0         100            -99  ...       None  NaN   
2         None  200      0         100            -99  ...       None  NaN   
3         None  200      0         100              1  ...       None  NaN   
4         None  200      0         100              1  ...       None  NaN   

   OFF3  CUT_BEGIN_FB  CUT_END_FB  EXTREMITY_TYPE2  NAME_LANG  INVER

In [3]:
class Graph:
    def __init__(self):
        self.adjacency_list = {}
        self.edge_lengths = {}

    def add_edge(self, v, w):
        if v not in self.adjacency_list:
            self.adjacency_list[v] = []
        if w not in self.adjacency_list:
            self.adjacency_list[w] = []
        if w not in self.adjacency_list[v] and v not in self.adjacency_list[w]:
            self.adjacency_list[v].append(w)
            self.adjacency_list[w].append(v)
            distance = self.calculate_distance(v, w)
            self.edge_lengths[(v, w)] = distance
            self.edge_lengths[(w, v)] = distance  

    def calculate_distance(self, start, end):
        return math.sqrt((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2)

    def get_adjacent(self, v):
        return self.adjacency_list.get(v, [])

    def get_edge_length(self, v, w):
        return self.edge_lengths.get((v, w), None)

    def __str__(self):
        return "Graph with nodes: " + str(list(self.adjacency_list.keys()))

    def display_adjacency_list(self):
        return self.adjacency_list

    def display_edge_lengths(self):
        return self.edge_lengths

# Create a graph
graph = Graph()

# Example DataFrame processing loop
for index, row in viz_roads.iterrows():  
    geometry = row['geometry']
    if isinstance(geometry, MultiLineString):
        for line in geometry.geoms:
            start_point = (line.coords[0][0], line.coords[0][1])
            end_point = (line.coords[-1][0], line.coords[-1][1])
            graph.add_edge(start_point, end_point)
    else:  # Single LineString
        start_point = (geometry.coords[0][0], geometry.coords[0][1])
        end_point = (geometry.coords[-1][0], geometry.coords[-1][1])
        graph.add_edge(start_point, end_point)

# Print the first 10 rows of the adjacency list
print("Adjacency List:")
for index, (key, value) in enumerate(graph.display_adjacency_list().items()):
    if index >= 10:
        break
    print(f"{key}: {value}")

# Print the first 10 rows of the edge lengths
print("Edge Lengths:")
for index, (key, length) in enumerate(graph.display_edge_lengths().items()):
    if index >= 10:
        break
    print(f"{key}: {length:.2f} units")


Adjacency List:
(2635432.1275000013, 1170007.039999999): [(2635453.1999999993, 1169889.1999999993), (2635524.2025000006, 1169944.8099999987)]
(2635453.1999999993, 1169889.1999999993): [(2635432.1275000013, 1170007.039999999), (2635547.719999999, 1169917.8237500004), (2635351.6712500006, 1169739.4262499996)]
(2635401.647500001, 1170101.65625): [(2635524.2025000006, 1169944.8099999987)]
(2635524.2025000006, 1169944.8099999987): [(2635401.647500001, 1170101.65625), (2635432.1275000013, 1170007.039999999), (2635547.719999999, 1169917.8237500004)]
(2635992.8337499984, 1170210.0287499987): [(2635883.401250001, 1169916.4475000016), (2636567.0862499997, 1170509.75), (2635862.446249999, 1170227.8099999987), (2636316.326250002, 1170622.7025000006)]
(2635883.401250001, 1169916.4475000016): [(2635992.8337499984, 1170210.0287499987), (2635775.8999999985, 1169816.6999999993)]
(2635582.1999999993, 1170101.4437500015): [(2635566.513750002, 1169921.7250000015), (2635424.9312499985, 1170175.7387499996),